In [51]:
import mediapipe as mp
import cv2
from PIL import Image
import numpy as np

tmp = 0 # 임의로
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

cap = cv2.VideoCapture(1)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()    
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)
        #print(results.face_landmarks)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )
        # Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )
        
        # 4. Pose Detections
        #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
        #                         mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
        #                         mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        #                         )
        
        nose_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x
        nose_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y
        #print(nose_x)
        #print(nose_y)
        
        # frame에 맞게 가이드라인 그려주기
        frame_width = cap.get(3)
        frame_height = cap.get(4)
        cv2.rectangle(image, (int(frame_width * 0.25), int(frame_height * 0.15)), (int(frame_width * 0.75), int(frame_height * 0.85))
                      , (255,0,0), 2)
        
 
        image_height, image_width = image.shape[:2]
        if results.right_hand_landmarks != None or results.left_hand_landmarks != None:
            tmp = 1
        
        # nose 기준 crop -> 웹캠에서 보이는 얼굴 크기에 따라 변동 있음
        '''
        if tmp == 0:
            cv2.imwrite('full_img1.jpg', image)
            img1 = Image.open('full_img1.jpg')
            cropimg = img1.crop((nose_x*image_width * 0.4, nose_y*image_height * 0, nose_x*image_width * 1.6, nose_y*image_height * 1.7))
            cropimg.save('crop_img1.jpg')
        '''
        
        # 가이드라인에 맞게 크롭, 간단
        if tmp == 1:
            cv2.imwrite('full_img1.jpg', image)
            img1 = Image.open('full_img1.jpg')
            cropimg = img1.crop((int(frame_width * 0.25), int(frame_height * 0.15), int(frame_width * 0.75), int(frame_height * 0.85)))
            cropimg.save('crop_img1.jpg')
        
        cv2.imshow('Webcam Feed', image)
        tmp = 0

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()